In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [3]:
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

In [5]:
# Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
)
''')

# Accounts table
cursor.execute('''
CREATE TABLE IF NOT EXISTS accounts (
    account_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    balance REAL DEFAULT 0.0,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
)
''')

# Transactions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    account_id INTEGER,
    type TEXT,
    amount REAL,
    timestamp TEXT,
    FOREIGN KEY(account_id) REFERENCES accounts(account_id)
)
''')
conn.commit()

In [7]:
# Insert customers
customers = [
    ('Alice', 'alice@example.com'),
    ('Bob', 'bob@example.com')
]
cursor.executemany('INSERT INTO customers (name, email) VALUES (?, ?)', customers)

# Create accounts for each customer
cursor.execute('INSERT INTO accounts (customer_id, balance) VALUES (1, 1000.0)')
cursor.execute('INSERT INTO accounts (customer_id, balance) VALUES (2, 500.0)')
conn.commit()

In [9]:
def deposit(account_id, amount):
    cursor.execute("UPDATE accounts SET balance = balance + ? WHERE account_id = ?", (amount, account_id))
    cursor.execute("INSERT INTO transactions (account_id, type, amount, timestamp) VALUES (?, 'deposit', ?, ?)",
                   (account_id, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()
    print(f"Deposited ${amount} into account {account_id}")

deposit(1, 200.0)

Deposited $200.0 into account 1


In [11]:
def withdraw(account_id, amount):
    cursor.execute("SELECT balance FROM accounts WHERE account_id = ?", (account_id,))
    balance = cursor.fetchone()[0]
    
    if balance >= amount:
        cursor.execute("UPDATE accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("INSERT INTO transactions (account_id, type, amount, timestamp) VALUES (?, 'withdraw', ?, ?)",
                       (account_id, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit()
        print(f"Withdrew ${amount} from account {account_id}")
    else:
        print("Insufficient balance")

withdraw(2, 100.0)

Withdrew $100.0 from account 2


In [13]:
def transfer(from_id, to_id, amount):
    cursor.execute("SELECT balance FROM accounts WHERE account_id = ?", (from_id,))
    from_balance = cursor.fetchone()[0]
    
    if from_balance >= amount:
        cursor.execute("UPDATE accounts SET balance = balance - ? WHERE account_id = ?", (amount, from_id))
        cursor.execute("UPDATE accounts SET balance = balance + ? WHERE account_id = ?", (amount, to_id))
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute("INSERT INTO transactions (account_id, type, amount, timestamp) VALUES (?, 'transfer-out', ?, ?)",
                       (from_id, amount, timestamp))
        cursor.execute("INSERT INTO transactions (account_id, type, amount, timestamp) VALUES (?, 'transfer-in', ?, ?)",
                       (to_id, amount, timestamp))
        conn.commit()
        print(f"Transferred ${amount} from account {from_id} to account {to_id}")
    else:
        print("Transfer failed: Insufficient balance")

transfer(1, 2, 150.0)

Transferred $150.0 from account 1 to account 2


In [15]:
def view_transactions(account_id):
    df = pd.read_sql_query("SELECT * FROM transactions WHERE account_id = ? ORDER BY timestamp DESC", conn, params=(account_id,))
    return df

view_transactions(1)

,transaction_id,account_id,type,amount,timestamp
0,3,1,transfer-out,150.0,2025-05-14 13:19:01
1,1,1,deposit,200.0,2025-05-14 13:18:38


In [17]:
conn.close()